### Preamble

In [1]:
import $ivy.`org.scalatest::scalatest:3.2.16`
import org.scalatest.{Filter => _, _}, flatspec._, matchers._

import $ivy.$                                

import org.scalatest.{Filter => _, _}, flatspec._, matchers._


# Topic 7. Applications

## 7.2 Implementing & querying data models

The Scala collections library greatly facilitate the implementation and querying of data models. For instance, the following classes model the structure of an organization which consists of departments, employees and tasks that employees can perform. 

In [2]:
// departments

case class Department(id: Department.Id/*, responsable: Employee.Id*/)
object Department:
    type Id = String

// tasks 

case class Task(id: Task.Id, hours: Int)
object Task:
    type Id = String

// Employees

case class Employee(id: Employee.Id, dpt: Department.Id)
object Employee:
    type Id = String

// The whole organization

case class Organization(
    departments: Map[Department.Id, Department], 
    tasks: Map[Task.Id, Task],
    employees: Map[Employee.Id, Employee], 
    knows: List[(Employee.Id, Task.Id)])

defined class Department
defined object Department
defined class Task
defined object Task
defined class Employee
defined object Employee
defined class Organization

In [7]:
def departments(org: Organization): List[Department] = 
    org.departments.values.toList

defined function departments

In [8]:
def departmentIds(org: Organization): List[Department.Id] = 
    org.departments.keys.toList

defined function departmentIds

In [9]:
departments(org)
departmentIds(org)

res9_0: List[Department] = List(
  Department(id = "Product"),
  Department(id = "Quality"),
  Department(id = "Research"),
  Department(id = "Sales")
)
res9_1: List[Id] = List("Product", "Quality", "Research", "Sales")

This implementation is an example of a _flat_ data model. The key feature of these kinds of models are that the different entities (employees, departments and tasks, in this case) refer to each other by using _keys_. This is a possible instance of the organization data model:

In [16]:
val org: Organization = Organization(
    Map(
        "Product"  -> Department("Product"),
        "Quality"  -> Department("Quality"),
        "Research" -> Department("Research"),
        "Sales"    -> Department("Sales")),
    
    Map("build"    -> Task("build", 3), 
        "abstract" -> Task("abstract", 5), 
        "design"   -> Task("design", 2),
        "call"     -> Task("call", 1),
        "program"  -> Task("program", 3)),
    
    Map("Alex"     -> Employee("Alex", "Product"), 
        "Bert"     -> Employee("Bert", "Product"), 
        "Cora"     -> Employee("Cora", "Research"), 
        "Drew"     -> Employee("Drew", "Research"), 
        "Edna"     -> Employee("Edna", "Research"), 
        "Fred"     -> Employee("Fred", "Sales")),
    
    List(
        ("Alex", "build"),
        ("Bert", "build"),
        ("Cora", "abstract"),
        ("Cora", "build"),
        ("Cora", "design"),
        ("Drew", "abstract"),
        ("Drew", "design"),
        ("Edna", "abstract"),
        ("Edna", "call"),
        ("Edna", "design"),
        ("Fred", "call")))

org: Organization = Organization(
  departments = Map(
    "Product" -> Department(id = "Product"),
    "Quality" -> Department(id = "Quality"),
    "Research" -> Department(id = "Research"),
    "Sales" -> Department(id = "Sales")
  ),
  tasks = HashMap(
    "program" -> Task(id = "program", hours = 3),
    "design" -> Task(id = "design", hours = 2),
    "abstract" -> Task(id = "abstract", hours = 5),
    "build" -> Task(id = "build", hours = 3),
    "call" -> Task(id = "call", hours = 1)
  ),
  employees = HashMap(
    "Alex" -> Employee(id = "Alex", dpt = "Product"),
    "Fred" -> Employee(id = "Fred", dpt = "Sales"),
    "Bert" -> Employee(id = "Bert", dpt = "Product"),
    "Cora" -> Employee(id = "Cora", dpt = "Research"),
    "Edna" -> Employee(id = "Edna", dpt = "Research"),
    "Drew" -> Employee(id = "Drew", dpt = "Research")
  ),
  knows = List(
    ("Alex", "build"),
    ("Bert", "build"),
    ("Cora", "abstract"),
    ("Cora", "build"),
    ("Cora", "design"),
    ("Drew", 

In [10]:
org.knows.filter(_._2 == "abstract").map(_._1)

res10: List[String] = List("Cora", "Drew", "Edna")

Flat data models are actually very close to the common _relational_ data models used in SQL persistent stores. This is the equivalent relational model of the organization database: 

![](../images/relational-model.png)

According to this mapping: 
- The `Organization` class represents the whole relational _database_.
- Members of this class correspond to the different _tables_ of the database, represented as `Map`s or simple `Set`s. We have four tables: the table of departments, employees, tasks, and a table which stores which tasks employees can perform.
- The key type of `Map` can be understood as the primary key of the relational table. The value type specifies the columns of the table. By convention, the identifier type is defined by the `Id` type alias in the companion object of the value type. For instance, the `employees` table is indexed by the employee identifier (a string value), and stores the department to which the employee belongs to.
- If the primary key consists of several keys, as in the `knows` table, we use tuples. 
- If the table just consists of the key (simple or composed) we use `Set` instead of `Map` (as the `knows` table also illustrates).



## Basic queries

Complex queries typically builds upon basic queries which are directly related to the structure of the data model. In particular, they are identified from the primary key and foreign-key relations in the relational model. In the organizational database we can identify the following queries:

In [11]:

object BasicQueries:

    // Entities
    
    def departments(org: Organization): List[Department] = 
        org.departments.values.toList

    def departmentIds(org: Organization): List[Department.Id] = 
        org.departments.keys.toList
    
    def getDepartment(id: Department.Id)(org: Organization): List[Department] = 
        org.departments.get(id).toList

    def employees(org: Organization): List[Employee] = 
        org.employees.values.toList

    def employeeIds(org: Organization): List[Employee.Id] = 
        org.employees.keys.toList

    def getEmployee(id: Employee.Id)(org: Organization): List[Employee] = 
        org.employees.get(id).toList

    // tasks, taskIds and getTask of the organization
    
    def tasks(org: Organization): List[Task] = 
        org.tasks.values.toList

    def taskIds(org: Organization): List[Task.Id] = 
        org.tasks.keys.toList

    def getTask(id: Task.Id)(org: Organization): List[Task] = 
        org.tasks.get(id).toList
    
    // 1-N relationships
    
    def employeeIds(dpt: Department.Id)(org: Organization): List[Employee.Id] = 
        org.employees.filter(_._2.dpt == dpt).map(_._1).toList
    
    // N-M relationships

    def capabilities(emp: Employee.Id)(org: Organization): List[Task.Id] = 
        org.knows.filter(_._1 == emp).map(_._2)
    
    // performerIds

    def performerIds(tid: Task.Id)(org: Organization): List[Employee.Id] = 
        org.knows.filter(_._2 == tid).map(_._1)

import BasicQueries._

defined object BasicQueries
import BasicQueries._


In [12]:
getTask("abstract")(org)
((getEmployee("Drew")(org): List[Employee]).map: 
    case Employee(id, dptId) => 
        getDepartment(dptId)(org): List[Department]): List[List[Department]]
((getEmployee("Drew")(org): List[Employee]).flatMap: 
    case Employee(id, dptId) => 
        getDepartment(dptId)(org): List[Department]): List[Department]

res12_0: List[Task] = List(Task(id = "abstract", hours = 5))
res12_1: List[List[Department]] = List(List(Department(id = "Research")))
res12_2: List[Department] = List(Department(id = "Research"))

In [13]:
// ¿Cuáles son todos los empleados del departmento de investigación?
employees(org).filter(??? : Employee => Boolean)

scala.NotImplementedError: an implementation is missing

In [13]:
employees(org).filter: 
    case Employee(id: String, dpt: String) => ???? : Boolean

-- [E006] Not Found Error: cell14.sc:2:46 --------------------------------------
2 |    case Employee(id: String, dpt: String) => ???? : Boolean
  |                                              ^^^^
  |                                              Not found: ????
  |
  | longer explanation available when compiling with `-explain`
Compilation Failed

In [14]:
employees(org).filter: 
    case Employee(id: String, dpt: String) => 
        dpt == "Research" : Boolean

res14: List[Employee] = List(
  Employee(id = "Cora", dpt = "Research"),
  Employee(id = "Edna", dpt = "Research"),
  Employee(id = "Drew", dpt = "Research")
)

## Sample queries

__Which are the tasks of the organization which can't be performed by any employee?__

In [15]:
class TestImpossibleTasks(
    impossibleTasks: Organization => List[Task.Id]
) extends AnyFlatSpec with should.Matchers:
    
    "impossibleTasks" should "work" in:
        impossibleTasks(org) shouldBe 
            List("program")

defined class TestImpossibleTasks

This is a conventional imperative implementation, using mutable variables:

In [61]:
taskIds(org).filter: 
    case tid => ??? : Boolean

scala.NotImplementedError: an implementation is missing

In [65]:
performerIds("abstract")(org).isEmpty
performerIds("program")(org).isEmpty

res65_0: Boolean = false
res65_1: Boolean = true

In [66]:
taskIds(org).filter: 
    case tid => performerIds(tid)(org).isEmpty : Boolean

res66: List[Id] = List("program")

In [ ]:
import collection.mutable.ListBuffer

def impossibleTasks(org: Organization): List[Task.Id] =
    ???

In [ ]:
run(TestImpossibleTasks(impossibleTasks))

This works but it is not the _functional_ style. The following version is closer to what we are looking for:

In [53]:
tasks(org)

res53: List[Task] = List(
  Task(id = "program", hours = 3),
  Task(id = "design", hours = 2),
  Task(id = "abstract", hours = 5),
  Task(id = "build", hours = 3),
  Task(id = "call", hours = 1)
)

In [67]:
def impossibleTasks(org: Organization): List[Task.Id] =
    taskIds(org).filter(tid => performerIds(tid)(org).isEmpty)

defined function impossibleTasks

In [69]:
def impossibleTasks(org: Organization): List[Task.Id] =
    taskIds(org).filter(performerIds(_)(org).isEmpty)

defined function impossibleTasks

In [69]:
def impossibleTasks(org: Organization): List[Task.Id] =
    taskIds(org).filter(performerIds(_)(org).isEmpty)

defined function impossibleTasks

or with pattern matching syntax:

In [ ]:
def impossibleTasks(org: Organization): List[Task.Id] =
    ???

In [68]:
run(TestImpossibleTasks(impossibleTasks))

cell52$Helper$TestImpossibleTasks:
impossibleTasks
- should work


But we can do it even better. We will endorse the following implementation that uses hight-level set operations (`diff`) and HOFs (`map`):

In [74]:

taskIds(org) diff org.knows.map(_._2).distinct

res74: List[Id] = List("program")

In [75]:
def impossibleTasks(org: Organization): List[Task.Id] =
    taskIds(org) diff org.knows.map(_._2)

defined function impossibleTasks

In [76]:
run(TestImpossibleTasks(impossibleTasks))

cell52$Helper$TestImpossibleTasks:
impossibleTasks
- should work


Arguably, this implementation conveys the intent of the function more clearly. It's more _declarative_. Moreover, it is more reliable since it builds upon standard methods of the Scala library (`diff` and `map`). It's true that the imperative version is also easy to read, but this is only because this is such a very simple function. We will see later on more complex examples where the functional solution shines brighter. 

__Which tasks can be performed by the employees of a given department?__

In [19]:
employeeIds("Research")(org)

res19: List[Id] = List("Cora", "Edna", "Drew")

In [22]:
capabilities("Cora")(org)
capabilities("Edna")(org)
capabilities("Drew")(org)

res22_0: List[Id] = List("abstract", "build", "design")
res22_1: List[Id] = List("abstract", "call", "design")
res22_2: List[Id] = List("abstract", "design")

In [37]:
employeeIds("Research")(org)
    .map((emp: Employee.Id) => capabilities(emp)(org): List[Task.Id])

res37: List[List[Id]] = List(
  List("abstract", "build", "design"),
  List("abstract", "call", "design"),
  List("abstract", "design")
)

In [38]:
employeeIds("Research")(org)
    .map((emp: Employee.Id) => emp.size)

res38: List[Int] = List(4, 4, 4)

In [24]:
employeeIds("Research")(org)
    .map(emp => capabilities(emp)(org))
    .flatten

res24: List[String] = List(
  "abstract",
  "build",
  "design",
  "abstract",
  "call",
  "design",
  "abstract",
  "design"
)

In [26]:
employeeIds("Research")(org)
    .flatMap(emp => capabilities(emp)(org))
    .distinct

res26: List[String] = List("abstract", "build", "design", "call")

In [27]:
class TestAllTasks(
    allTasks: Department.Id => Organization => List[Task.Id]
) extends AnyFlatSpec with should.Matchers:

    "allTasks" should "work" in:
        allTasks("Product")(org).toSet shouldBe 
            Set("build")
        allTasks("Quality")(org).toSet shouldBe 
            Set()
        allTasks("Sales")(org).toSet shouldBe 
            Set("call")
        allTasks("Research")(org).toSet shouldBe 
            Set("abstract", "build", "design", "call")

defined class TestAllTasks

The basic queries of the data model allow us to obtain all the employees of an organization, and the tasks that they can perform. So, this a first step towards the solution:

In [28]:
// def allTasks(dpt: Department.Id)(org: Organization): List[Task.Id]
def allTasks(dpt: Department.Id)(org: Organization): List[List[Task.Id]] = 
    ???

defined function allTasks

However, this is not the signature that we need to implement, since we are returning a set of sets of tasks, not a set of tasks. In order to do it right we need also to _flatten_ the result, i.e. concatenate all the individual sets of tasks for each employee. In sum, we need the `flatMap` HOF:

In [29]:
def allTasks(dpt: Department.Id)(org: Organization): List[Task.Id] = 
    employeeIds(dpt)(org)
        .flatMap(emp => capabilities(emp)(org))
        .distinct

defined function allTasks

In [30]:
run(TestAllTasks(allTasks))

cell27$Helper$TestAllTasks:
allTasks
- should work


__Compute the list of departments of an organization together with the number of tasks their employees can perform, sorted by the number of tasks__

In [47]:
class TestSortedDeps(
    sortedDeps: Organization => List[(Department.Id, Int)]
) extends AnyFlatSpec with should.Matchers:
    
    "sortedDeps" should "work" in:
        sortedDeps(org) shouldBe 
            List(("Research",4), ("Product",1), ("Sales",1), ("Quality",0))

defined class TestSortedDeps

In [49]:
Set(1,2,3) == Set(3,2,1)
List(1,2,3) == List(3,2,1)

res49_0: Boolean = true
res49_1: Boolean = false

In [32]:
departmentIds(org)

res32: List[Id] = List("Product", "Quality", "Research", "Sales")

In [35]:
allTasks("Product")(org).size
allTasks("Quality")(org).size
allTasks("Research")(org).size
allTasks("Sales")(org).size


res35_0: Int = 1
res35_1: Int = 0
res35_2: Int = 4
res35_3: Int = 1

In [55]:
departmentIds(org)
    .map: (dpt: Department.Id) =>
        (dpt : Department.Id, allTasks(dpt)(org).size : Int)

res55: List[(String, Int)] = List(
  ("Product", 1),
  ("Quality", 0),
  ("Research", 4),
  ("Sales", 1)
)

In [55]:
departmentIds(org)
    .map: (dpt: Department.Id) =>
        (dpt : Department.Id, allTasks(dpt)(org).size : Int)
    .sortWith(??? : ((Department.Id, Int), (Department.Id, Int)) => Boolean)

res55: List[(String, Int)] = List(
  ("Product", 1),
  ("Quality", 0),
  ("Research", 4),
  ("Sales", 1)
)

In [58]:
departmentIds(org)
    .map: (dpt: Department.Id) =>
        (dpt : Department.Id, allTasks(dpt)(org).size : Int)
    .sortWith((t1: (Department.Id, Int), t2: (Department.Id, Int)) => 
        t1._2 < t2._2 : Boolean)
    .reverse

res58: List[(String, Int)] = List(
  ("Research", 4),
  ("Sales", 1),
  ("Product", 1),
  ("Quality", 0)
)

In [59]:
departmentIds(org)
    .map: (dpt: Department.Id) =>
        (dpt : Department.Id, allTasks(dpt)(org).size : Int)
    .sortWith((t1: (Department.Id, Int), t2: (Department.Id, Int)) => 
        t1._2 > t2._2 : Boolean)

res59: List[(String, Int)] = List(
  ("Research", 4),
  ("Product", 1),
  ("Sales", 1),
  ("Quality", 0)
)

In [60]:
def sortedDeps(org: Organization): List[(Department.Id, Int)] = 
    departmentIds(org)
        .map: (dpt: Department.Id) =>
            (dpt : Department.Id, allTasks(dpt)(org).size : Int)
        .sortWith((t1: (Department.Id, Int), t2: (Department.Id, Int)) => 
            t1._2 > t2._2 : Boolean)

defined function sortedDeps

In [62]:
def sortedDeps(org: Organization): List[(Department.Id, Int)] = 
    departmentIds(org)
        .map: (dpt: Department.Id) =>
            (dpt : Department.Id, allTasks(dpt)(org).size : Int)
        .sortWith:  
            case ((_, s1), (_, s2)) => s1 > s2 

defined function sortedDeps

or with pattern matching syntax:

In [ ]:
def sortedDeps(org: Organization): List[(Department.Id, Int)] = 
    ???

In [63]:
run(TestSortedDeps(sortedDeps))

cell47$Helper$TestSortedDeps:
sortedDeps
- should work


In [ ]:
List(1,6,2,3,5,0).sortWith(??? : (Int, Int) => Boolean)

In [52]:
List(1,6,2,3,5,0).sortWith((i1: Int, i2: Int) => i1 < i2 : Boolean)

res52: List[Int] = List(0, 1, 2, 3, 5, 6)

In [53]:
List(1,6,2,3,5,0).sortWith(_ < _)

res53: List[Int] = List(0, 1, 2, 3, 5, 6)

In [54]:
List(1,6,2,3,5,0).sortBy(i => i)

res54: List[Int] = List(0, 1, 2, 3, 5, 6)

__Which are the employees who can perform tasks of a given duration?__

In [79]:
class TestPersistentEmps(
    persistentEmps: Int => Organization => List[Employee.Id]
) extends AnyFlatSpec with should.Matchers:
    
    "persistentEmps" should "work" in:
        persistentEmps(3)(org).toSet shouldBe 
            Set("Cora", "Drew", "Edna")

defined class TestPersistentEmps

In [64]:
employeeIds(org)
    .filter(??? : Employee.Id => Boolean)

res64: List[Id] = List("Alex", "Fred", "Bert", "Cora", "Edna", "Drew")

In [68]:
capabilities("Cora")(org)

res68: List[Id] = List("abstract", "build", "design")

In [70]:

getTask("abstract")(org)
getTask("build")(org)
getTask("design")(org)

res70_0: List[Task] = List(Task(id = "abstract", hours = 5))
res70_1: List[Task] = List(Task(id = "build", hours = 3))
res70_2: List[Task] = List(Task(id = "design", hours = 2))

In [71]:
capabilities("Cora")(org)
    .map(tid => getTask(tid)(org))

res71: List[List[Task]] = List(
  List(Task(id = "abstract", hours = 5)),
  List(Task(id = "build", hours = 3)),
  List(Task(id = "design", hours = 2))
)

In [72]:
capabilities("Cora")(org)
    .flatMap(tid => getTask(tid)(org))

res72: List[Task] = List(
  Task(id = "abstract", hours = 5),
  Task(id = "build", hours = 3),
  Task(id = "design", hours = 2)
)

In [76]:
capabilities("Cora")(org)
    .flatMap(tid => getTask(tid)(org))
    .filter(t => t.hours > 3)
    .nonEmpty

res76: Boolean = true

In [77]:
employeeIds(org)
    .filter: emp => 
        capabilities(emp)(org)
            .flatMap(tid => getTask(tid)(org))
            .filter(t => t.hours > 3)
            .nonEmpty

res77: List[Id] = List("Cora", "Edna", "Drew")

In [73]:
Task("abstract", 5).hours > 3

res73: Boolean = true

In [80]:
def persistentEmps(min: Int)(org: Organization): List[Employee.Id] = 
    employeeIds(org)
        .filter: emp => 
            capabilities(emp)(org)
                .flatMap(tid => getTask(tid)(org))
                .filter(t => t.hours > min)
                .nonEmpty

defined function persistentEmps

In [87]:
def persistentEmps(min: Int)(org: Organization): List[Employee.Id] = 
    employeeIds(org)
        .filter: emp => 
            capabilities(emp)(org)
                .flatMap(tid => getTask(tid)(org))
                .exists(t => t.hours > min)

defined function persistentEmps

In [83]:
List(1,2,3,4,5).filter(_ % 2 == 0).nonEmpty

res83: Boolean = true

In [85]:
List(1,2,3,4,5).exists(_ % 2 == 0)

res85: Boolean = true

In [84]:
List(1,3,5).filter(_ % 2 == 0).nonEmpty

res84: Boolean = false

In [86]:
List(1,3,5).forall(_ % 2 != 0)

res86: Boolean = true

Alterrnatively, we can also bet by with `filter` and `exists`:

In [ ]:
def persistentEmps(min: Int)(org: Organization): List[Employee.Id] = 
    ???

In [88]:
run(TestPersistentEmps(persistentEmps))

cell79$Helper$TestPersistentEmps:
persistentEmps
- should work


__Which are the departments whose employees, as a team, know how to perform a given set of tasks?__

In [ ]:
class TestDptsThatKnowHowTo(
    dptsThatKnowHowTo: Set[Task.Id] => Organization => List[Department.Id]
) extends AnyFlatSpec with should.Matchers:
    
    "dptsThatKnowHowTo" should "work" in:
        dptsThatKnowHowTo(Set())(org).toSet shouldBe 
            Set("Sales", "Product", "Quality", "Research")
        dptsThatKnowHowTo(Set("call"))(org).toSet shouldBe 
            Set("Sales", "Research")
        dptsThatKnowHowTo(Set("call", "abstract"))(org).toSet shouldBe 
            Set("Research")

We can build upon the previous function `allTasks`:

In [ ]:
def dptsThatKnowHowTo(tasks: Set[Task.Id])(org: Organization): List[Department.Id] = 
    ???

In [ ]:
run(TestDptsThatKnowHowTo(dptsThatKnowHowTo))

__Obtain a list of employees sorted by the number of tasks that they can perform__

In [ ]:
class TestSortedEmployees(
    sortedEmployees: Organization => List[(Employee.Id, Int)]
) extends AnyFlatSpec with should.Matchers:
    
    "sortedEmployees" should "work" in:
        sortedEmployees(org) shouldBe 
            List(
              ("Alex", 1),
              ("Fred", 1),
              ("Bert", 1),
              ("Drew", 2),
              ("Cora", 3),
              ("Edna", 3))

We may attempt the following:

In [ ]:
def sortedEmployees(org: Organization): List[(Employee.Id, Int)] = 
    ???

and this is almost right: we are missing those employees that can't perform any task. This is the right one:

In [ ]:
def sortedEmployees(org: Organization): List[(Employee.Id, Int)] = 
    ???

In [ ]:
run(TestSortedEmployees(sortedEmployees))

__Which are the departments whose employees are all able to perform a given task?__

In [ ]:
class TestExpertDepsIn(
    expertDpts: Task.Id => Organization => List[Department.Id]
) extends AnyFlatSpec with should.Matchers:
    
    "expertDpts" should "work" in:
        expertDpts("abstract")(org).toSet shouldBe 
            Set("Quality", "Research")

The conventional imperative solution is quite complex: 

In [ ]:
def expertDepsIn(task: Task.Id)(org: Organization): List[Department.Id] =
    ???

In [ ]:
run(new TestExpertDepsIn(expertDepsIn))

This is not only more complex to understand, but prone to error. In order to obtain a simpler (and functional) solution by first declaring in plain natural language the intended query:

In [ ]:
def expertDepsIn(tsk: Task.Id)(org: Organization): List[Department.Id] = 
    // From all the departments of the organization, choose
    // those that for all its employees
    // the specified task is included in their capabilities
    ???

Then, we can formalize the natural language specification by relying on standard HOFs (`filter`, `forall`) and collection operations (`contains`):

In [ ]:
def expertDepsIn(tsk: Task.Id)(org: Organization): List[Department.Id] = 
    ???

In [ ]:
run(TestExpertDepsIn(expertDepsIn))